In [9]:
import pandas as pd

df = pd.read_excel("Healthcare dataset.xlsx")
df.head()


,date,patient_id,patient_gender,patient_age,patient_sat_score,Full Name,patient_race,patient_admin_flag,patient_waittime,department_referral,Moment
0,2019-04-01,874-57-8352,M,38,NaN,V Matyushenko,White,False,50,NaN,AM
1,2019-04-01,259-51-9518,M,62,NaN,T Rolingson,Asian,False,12,NaN,AM
2,2019-04-01,417-42-4910,F,79,NaN,V Clelle,Two or More Races,False,45,NaN,PM
3,2019-04-01,428-40-4491,F,46,NaN,J Antonnikov,African American,True,45,NaN,PM
4,2019-04-01,868-10-5414,F,55,NaN,R Fulker,African American,True,24,NaN,PM


In [16]:
df["patient_sat_score"].isna().mean() #This tells % missing.


np.float64(0.0)

In [11]:
median_score = df["patient_sat_score"].median()
df["patient_sat_score"] = df["patient_sat_score"].fillna(median_score)


Fill missing with the median

Why?
Not affected by extreme ratings....
Keeps realistic central tendency

In [17]:
df["department_referral"] = df["department_referral"].fillna("Walk-In")


In [19]:
df.isna().sum().sum()


np.int64(0)

In [12]:
## Dataset Overview
df.info()
df.describe()



<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9216 entries, 0 to 9215
Data columns (total 11 columns):
 #   Column               Non-Null Count  Dtype         
---  ------               --------------  -----         
 0   date                 9216 non-null   datetime64[ns]
 1   patient_id           9216 non-null   object        
 2   patient_gender       9216 non-null   object        
 3   patient_age          9216 non-null   int64         
 4   patient_sat_score    9216 non-null   float64       
 5   Full Name            9216 non-null   object        
 6   patient_race         9216 non-null   object        
 7   patient_admin_flag   9216 non-null   bool          
 8   patient_waittime     9216 non-null   int64         
 9   department_referral  3816 non-null   object        
 10  Moment               9216 non-null   object        
dtypes: bool(1), datetime64[ns](1), float64(1), int64(2), object(6)
memory usage: 729.1+ KB


,date,patient_age,patient_sat_score,patient_waittime
count,9216,9216.000000,9216.000000,9216.000000
mean,2020-01-16 02:22:48.749999872,39.855143,4.997830,35.259874
min,2019-04-01 00:00:00,1.000000,0.000000,10.000000
25%,2019-08-24 00:00:00,20.000000,5.000000,23.000000
50%,2020-01-17 00:00:00,39.000000,5.000000,35.000000
75%,2020-06-09 00:00:00,60.000000,5.000000,48.000000
max,2020-10-30 00:00:00,79.000000,10.000000,60.000000
std,NaN,22.755125,1.639712,14.735323


This section inspects data types, missing values, and basic statistics.


In [20]:

df.head()


,date,patient_id,patient_gender,patient_age,patient_sat_score,Full Name,patient_race,patient_admin_flag,patient_waittime,department_referral,Moment
0,2019-04-01,874-57-8352,M,38,5.0,V Matyushenko,White,False,50,Walk-In,AM
1,2019-04-01,259-51-9518,M,62,5.0,T Rolingson,Asian,False,12,Walk-In,AM
2,2019-04-01,417-42-4910,F,79,5.0,V Clelle,Two or More Races,False,45,Walk-In,PM
3,2019-04-01,428-40-4491,F,46,5.0,J Antonnikov,African American,True,45,Walk-In,PM
4,2019-04-01,868-10-5414,F,55,5.0,R Fulker,African American,True,24,Walk-In,PM


In [29]:
## Which departments have the longest patient wait times?
df.groupby("department_referral")["patient_waittime"].mean().sort_values(ascending=False)


,patient_waittime
department_referral,
Neurology,36.803109
Physiotherapy,36.565217
Gastroenterology,35.831461
Cardiology,35.354839
Walk-In,35.293148
Orthopedics,34.982915
General Practice,34.912500
Renal,34.697674


Long waiting times reduce satisfaction and increase patient complaints.
ans:- Neurology and Physiotherapy have the highest average patient wait times

In [30]:
### Does waiting time impact patient satisfaction?
df.groupby(pd.cut(df["patient_waittime"], bins=[0,30,60,120]))["patient_sat_score"].mean()



/tmp/ipython-input-3793658176.py:2: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  df.groupby(pd.cut(df["patient_waittime"], bins=[0,30,60,120]))["patient_sat_score"].mean()


,patient_sat_score
patient_waittime,
"(0, 30]",5.010670
"(30, 60]",4.989025
"(60, 120]",NaN


Patient satisfaction decreases when wait time increases from under 30 minutes to 30–60 minutes (5.04 → 4.96). No patients waited longer than 60 minutes, indicating the hospital is effectively controlling excessive delays.

In [26]:
#Which age group is least satisfied?”
df.groupby(pd.cut(df["patient_age"], bins=[0,18,35,50,65,100]))["patient_sat_score"].mean()


/tmp/ipython-input-2552599991.py:2: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  df.groupby(pd.cut(df["patient_age"], bins=[0,18,35,50,65,100]))["patient_sat_score"].mean()


,patient_sat_score
patient_age,
"(0, 18]",4.990995
"(18, 35]",5.003526
"(35, 50]",5.051239
"(50, 65]",5.015625
"(65, 100]",4.922078


Older patients (65+) report lower satisfaction, suggesting accessibility or care quality issues.

In [31]:
#Is hospital workload heavier in AM or PM?”
df["Moment"].value_counts(normalize=True) * 100

,proportion
Moment,
AM,50.260417
PM,49.739583


Hospital visits are almost equally split between AM (50.26%) and PM (49.74%), indicating a balanced workload across the day with a marginally higher demand in the morning.

In [32]:
#Which departments attract the most satisfied patients?
df.dropna(subset=["patient_sat_score"]) \
  .groupby("department_referral")["patient_sat_score"] \
  .mean() \
  .sort_values(ascending=False)



,patient_sat_score
department_referral,
Gastroenterology,5.241573
Neurology,5.077720
Cardiology,5.040323
General Practice,5.016848
Physiotherapy,4.996377
Walk-In,4.987593
Orthopedics,4.958794
Renal,4.883721


Among patients who provided feedback, Gastroenterology and Neurology show the highest satisfaction scores, while Renal and Orthopedics lag behind, indicating potential need for service quality and wait-time improvements in those departments.